In [53]:
import pandas as pd

In [54]:
movie_df = pd.read_csv("../backend/data/movies_2020_new.csv").drop(columns=["isAdult", "startYear", "runtimeMinutes", "posterPath", "backdropPath", "trailerPath"])
movie_df["description"] = movie_df["description"].str.replace("\n", "", regex=True)
movie_df["description"] = movie_df["description"].str.replace("\r", "", regex=True)
movie_df["description"] = movie_df["description"].str.replace("\t", "", regex=True)

In [55]:
principal_df = pd.read_csv("../backend/data/new_title_principals.csv").drop(columns=["ordering", "job", "characters"])
drop_idx = principal_df[principal_df["category"].isin(["self", "producer", "director", "writer"])].index
principal_df.drop(drop_idx, inplace=True)
principal_df.drop(columns=["category"], inplace=True)
principal_df.head(10)

,tconst,nconst
0,tt10054536,nm0079664
1,tt10054536,nm5998366
6,tt10095582,nm0000243
7,tt10095582,nm0000531
8,tt10095582,nm1307435
9,tt10095582,nm1584487
10,tt10095582,nm0322407
11,tt10095582,nm3659660
12,tt10095582,nm0577982
13,tt10095582,nm0027215


In [56]:
movie_principal_df = movie_df.merge(principal_df, how="left", left_on="tconst", right_on="tconst")
movie_principal_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,genres,description,nconst
0,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,nm0031652
1,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,nm0250185


In [57]:
name_df = pd.read_csv("../backend/data/new_name_basics.csv").drop(columns=["primaryProfession", "birthYear", "deathYear", "knownForTitles"])
name_df.head()

,nconst,primaryName
0,nm0000040,Stanley Kubrick
1,nm0000054,Marilyn Monroe
2,nm0000087,Elena Koreneva
3,nm0000093,Brad Pitt
4,nm0000119,Tia Carrere


In [58]:
movie_principal_name_df = movie_principal_df.merge(name_df, how="inner", left_on="nconst", right_on="nconst")
movie_principal_name_df.drop(columns=["nconst"], inplace=True)
movie_principal_name_df.head()

,tconst,titleType,primaryTitle,originalTitle,genres,description,primaryName
0,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,Nemesio Antúnez
1,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,Marcial Edwards
2,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,Colectivo Obrero Elmo Catalán
3,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,Javier Maldonado
4,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,Rodrigo Maturana


In [59]:
# group by tconst and insert a list of all the names
movie_name = movie_principal_name_df.groupby("tconst").agg({"primaryName": lambda x: list(x)}).reset_index()
movie_principal_name_df = movie_df.merge(movie_name, how="left", left_on="tconst", right_on="tconst")
movie_principal_name_df

,tconst,titleType,primaryTitle,originalTitle,genres,description,primaryName
0,tt0070596,movie,Socialist Realism,El realismo socialista,"['Drama', 'History']",A satirical take on President Salvador Allende...,"[Nemesio Antúnez, Marcial Edwards, Colectivo O..."
1,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,['Documentary'],NaN,NaN
2,tt0104988,movie,Neues in Wittstock,Neues in Wittstock,['Documentary'],Sixth Wittstock film.,NaN
3,tt0120589,movie,The Surgeon of the Rusty Knife,Predestinado,"['Biography', 'Drama']","José Pedro de Freitas, better known as Zé Arig...","[Danton Mello, Juliana Paes, Marcos Caruso, Cá..."
4,tt0164115,movie,Nine Ball,Nine Ball,"['Drama', 'Fantasy']",For years Nicky has been drinking to try and f...,"[Kenny Johnson, Rich Grosso, Allen Schneider, ..."
...,...,...,...,...,...,...,...
58805,tt9898844,movie,The Hunting,The Hunting,"['Horror', 'Mystery', 'Thriller']",When a mysterious animal attack leaves a mutil...,"[Angela Cole, Joelle Westwood, Alan Tuskes, Ke..."
58806,tt9900940,movie,Scrapper,The Scrapper,"['Crime', 'Drama', 'Thriller']","Ex-con turned ""scrapper"" Jake leads a quiet li...","[Myles Clohessy, Craig muMs Grant, Daniel Danc..."
58807,tt9904648,movie,The Contrast,The Contrast,['Comedy'],On the day before her wedding to a hipster pla...,"[Joy Villa, Jermain Hollman, Lee Donoghue, Dea..."
58808,tt9907782,movie,The Cursed,Eight for Silver,"['Fantasy', 'Horror', 'Mystery']","In the late 19th century, a brutal land baron ...","[Boyd Holbrook, Kelly Reilly, Alistair Petrie,..."


In [ ]:
def get_text(row):
    return str(row["tconst"]) + "_" + str(row["genres"]) + "_" + str(row["description"]) + "_" + str(row["primaryName"])
movie_principal_name_df["text"] = movie_principal_name_df.apply(get_text, axis=1)


In [61]:
movie_df.shape

(58810, 6)

In [62]:
movie_principal_name_df.shape

(58810, 8)

In [63]:
f = open("../backend/data/movies_descriptions.txt", "w")
for idx in range(0, len(movie_principal_name_df)):
    f.write(movie_principal_name_df.iloc[idx]["text"] + "\n")
f.close()